Ground Truth Extraction

In [ ]:
import pandas as pd
df = pd.read_csv("df.csv")

In [ ]:
import os
from openai import OpenAI

# --- setup ---
os.environ['OPENAI_API_KEY'] = 'YOUR_KEY_HERE'
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

def stream_chat(model, messages, temperature=0):
    """Return a single concatenated string from a streaming chat completion."""
    out = []
    stream = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        stream=True,
    )
    for chunk in stream:
        out.append(chunk.choices[0].delta.content or "")
    return "".join(out).strip()

# =============== 1) EXTRACT FROM df['Limitation'] =================
generated_summary = []
for i in range(len(df)):
    src = str(df.at[i, 'Limitation']) if 'Limitation' in df.columns else ""
    prompt = (
        "Here is a limitation. Extract sub limitation from this limitation. Each sub limitation contains separate topics, limitation or point. "
        f"{src}"
    )
    text = stream_chat(
        "gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    generated_summary.append(text)

df["limitation_extracted"] = generated_summary  # NEW COLUMN

# =============== 2) EXTRACT FROM df['OpenReview'] =================
generated_summary1 = []
for i in range(len(df)):
    src = str(df.at[i, 'OpenReview']) if 'OpenReview' in df.columns else ""
    prompt = (
        "Here is bunch of reviews. Extract review and each review contains separate topics, limitation or point.\n\n"
        f"{src}"
    )
    text = stream_chat(
        "gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    generated_summary1.append(text)

df["openreview_extracted"] = generated_summary1  # NEW COLUMN

# =============== 3) MERGE THE TWO EXTRACTIONS =====================
merged_texts = []
for i in range(len(df)):
    A = str(df.at[i, "limitation_extracted"] or "")
    B = str(df.at[i, "openreview_extracted"] or "")

    merge_prompt = f"""
You will receive two texts, A and B, each already extracted as bullet points.

Rules (very important):
- Merge ONLY by copying exact sentences/points from A and/or B.
- Do NOT paraphrase, summarize, reword, or invent anything.
- If A and B contain the same/similar points, deduplicate and output a single merged bullet list.
- If they are different, output a single bullet list that contains all distinct points from A and B (still copied verbatim).
- Preserve original wording. If one is empty, just return the other.
- Output only the bullet list, nothing else.

Text A:
{A}

Text B:
{B}
"""
    merged = stream_chat(
        "gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "You are a meticulous editor who never adds new content and only copies verbatim items from provided texts."
            },
            {"role": "user", "content": merge_prompt}
        ],
        temperature=0
    )
    merged_texts.append(merged)

df["merged_limitation"] = merged_texts  # FINAL MERGED COLUMN
